## Подготовка данных для занятия

In [1]:
import re

import pandas as pd
import pymorphy2


In [102]:
df_origin = pd.read_csv("content_description.csv", sep='\t')
df_origin.head(1)

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."


In [103]:
df = df_origin.drop('content', axis=1)
df.head(1)

,description
0,"Лучший подарок, который только можно было прид..."


In [104]:
reg_ex = r'\w+'
reg_ex_compiled = re.compile(reg_ex)
func = lambda x: reg_ex_compiled.findall(x.lower())
df['description'] = df['description'].apply(func)
df.head(1)

,description
0,"[лучший, подарок, который, только, можно, было..."


In [105]:
morph = pymorphy2.MorphAnalyzer()
def func(x: list) -> pd.Series:
    x_ser = pd.Series(x)
    func_in = lambda el: morph.parse(el)[0].normal_form
    return x_ser.apply(func_in)
df = df['description'].apply(func)
df

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,хороший,подарок,который,только,можно,быть,придумать,для,весь,поклонник,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,через,какой,трудность,приходиться,проходить,сант,клаус,каждый,год,чтобы,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,миловидный,давид,позор,для,свой,отец,не,в,сила,терпеть,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,экранизация,сатирический,бестселлер,стивен,фрая,мастерски,воплотить,британский,режиссёр,и,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,леди,удача,авантюрный,романтический,комедия,снятой,в,двадцатый,год,хх,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,на,поминки,сюзанна,узнать,что,джефф,изменять,она,с,молодой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,культовый,японский,фильм,ужас,заставлять,кровь,стыть,в,жила,и,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,двое,выпускник,калифорнийский,школа,бизнес,ставить,перед,себя,простой,и,...,реальный,событие,клуб,миллиардер,можно,смотреть,онлайн,на,наш,сайт


In [106]:
df = df.T.unstack().reset_index().drop('level_1', axis=1).rename({0: 'word', 'level_0': 'doc_id'}, axis=1)
df.tail(5)

,doc_id,word
1435,7,смотреть
1436,7,онлайн
1437,7,на
1438,7,наш
1439,7,сайт


In [107]:
df[df.doc_id == 0].tail()

,doc_id,word
175,0,NaN
176,0,NaN
177,0,NaN
178,0,NaN
179,0,NaN


In [108]:
df.dropna(subset=['word'], inplace=True)

In [109]:
df[df.doc_id == 0].tail()

,doc_id,word
151,0,спецвыпуск
152,0,шерлок
153,0,холмс
154,0,безобразный
155,0,невеста


In [110]:
df['dummy'] = 1
df.head()

,doc_id,word,dummy
0,0,хороший,1
1,0,подарок,1
2,0,который,1
3,0,только,1
4,0,можно,1


In [111]:
df_word_cnt = df.groupby(['doc_id', 'word']).agg(word_cnt = ('dummy', 'count')).reset_index()
df_word_cnt.head()

,doc_id,word,word_cnt
0,0,1895,1
1,0,а,1
2,0,абсолютно,1
3,0,англия,1
4,0,безобразный,1


In [112]:
df_word_cnt.sort_values(['doc_id', 'word_cnt'], ascending=(True, False)).head(10)

,doc_id,word,word_cnt
6,0,в,10
29,0,и,7
37,0,который,5
77,0,с,4
84,0,сериал,4
111,0,шерлок,4
75,0,риколетти,3
5,0,быть,2
8,0,весь,2
21,0,для,2


In [4]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
28,0,и,7
36,0,который,5
110,0,шерлок,4
83,0,сериал,4
76,0,с,4
8,0,весь,3
74,0,риколетти,3
104,0,холмс,2
22,0,же,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [154]:
doc_count_df = df.groupby(['word', 'doc_id']).agg(lambda x: 1).reset_index()
doc_count_df.head()

,word,doc_id,dummy
0,1895,0,1
1,1980,7,1
2,1983,7,1
3,2,1,1
4,kingsman,7,1


In [155]:
doc_count = doc_count_df.doc_id.unique().size
print(doc_count)
doc_count_df = doc_count_df.groupby('word').agg(doc_cnt=('doc_id', 'count')) * 100 / doc_count
doc_count_df.sort_values('doc_cnt', ascending=False)

8


,doc_cnt
word,
и,100.0
в,100.0
на,87.5
с,87.5
он,75.0
...,...
лишаться,12.5
лос,12.5
любить,12.5


In [8]:
# -- ВАШ КОД ЗДЕСЬ --

doc_frequency = pd.DataFrame()

doc_frequency.head(10)

""


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [156]:
df_origin.head()

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...


In [158]:
text = df_origin.iloc[3,1]
text

'Экранизация сатирического бестселлера Стивена Фрая, мастерски воплощенная британским режиссером и продюсером Джоном Дженксом.   У Теда Уоллеса было славное прошлое, и вот-вот его книги должны были пополнить стройный ряд томов британских классиков. Но его жуткий характер вкупе с алкоголизмом и мизантропией в запущенной стадии испортили все. Теперь Тед зарабатывает на жизнь тем, что поносит в своих статьях спектакли, которые вообще-то не стоят его внимания. И продолжает пить. Вскоре Тед лишается и этого: его бесконечное ворчание достало всю редакцию.  Готовясь к голодной и бесславной смерти, внезапно Уоллес получает выгодное предложение от своей крестницы Джейн. Девушка недавно излечилась от болезни благодаря чуду и теперь предлагает Теду погостить в аристократическом доме, чтобы осветить происходящие здесь чудеса исцеления.  Старый скептик соглашается из корыстных побуждений и даже представить не может, сколько сюрпризов ему уготовано.  Смотреть онлайн эту комедию о том, что даже самые

In [203]:
reg_ex = r'[А-Я]{1}\w+ [А-Я]{1}\w+'
re.findall(reg_ex, text)

['Стивена Фрая', 'Джоном Дженксом', 'Теда Уоллеса', 'Теперь Тед', 'Вскоре Тед']

In [6]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'\. \w+'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

. Но
. Теперь
. И
. Вскоре
. Девушка


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [205]:
df_word_cnt.head()

,doc_id,word,word_cnt
0,0,1895,1
1,0,а,1
2,0,абсолютно,1
3,0,англия,1
4,0,безобразный,1


In [206]:
genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}


In [210]:
import numpy as np

In [213]:
genre_list = []
for k, v in genre_dict.items():
    for w in v:
        genre_list.append([k, w])
genre_list

[['комедия', 'сатирический'],
 ['комедия', 'авантюрный'],
 ['комедия', 'забавный'],
 ['мелодрама', 'выбор'],
 ['мелодрама', 'позор'],
 ['сказка', 'приключения'],
 ['сказка', 'милый'],
 ['сказка', 'семейный'],
 ['детектив', 'тайна'],
 ['детектив', 'разгадать'],
 ['детектив', 'загадочный'],
 ['триллер', 'ужас'],
 ['триллер', 'зловещий'],
 ['триллер', 'нерв']]

In [231]:
genre_df = pd.DataFrame(genre_list, columns = ['genre', 'character_word'])
genre_df

,genre,character_word
0,комедия,сатирический
1,комедия,авантюрный
2,комедия,забавный
3,мелодрама,выбор
4,мелодрама,позор
5,сказка,приключения
6,сказка,милый
7,сказка,семейный
8,детектив,тайна
9,детектив,разгадать


In [257]:
df_genre = df_word_cnt.drop('word_cnt', axis=1).merge(genre_df, left_on='word', right_on='character_word', how='inner').drop('character_word', axis=1)
df_genre

,doc_id,word,genre
0,0,загадочный,детектив
1,0,разгадать,детектив
2,0,тайна,детектив
3,0,ужас,триллер
4,6,ужас,триллер
5,1,милый,сказка
6,2,выбор,мелодрама
7,2,позор,мелодрама
8,3,сатирический,комедия
9,4,авантюрный,комедия


In [258]:
df_genre = df_genre.groupby(['doc_id', 'genre']).agg(lambda x: 1).drop('word', axis=1).reset_index()
df_genre

,doc_id,genre
0,0,детектив
1,0,триллер
2,1,сказка
3,2,мелодрама
4,3,комедия
5,4,комедия
6,6,триллер


In [262]:
def func(x):
    return list(x)
df_genre.groupby('doc_id')['genre'].agg(func).to_frame().reset_index()

,doc_id,genre
0,0,"[детектив, триллер]"
1,1,[сказка]
2,2,[мелодрама]
3,3,[комедия]
4,4,[комедия]
5,6,[триллер]
